In [8]:
import subprocess
import footsteps
import os
import glob
from PIL import Image
import numpy as np
import tqdm

def DavisEval(model, name):
    davis_path = "data_storage/DAVIS/"
    output_path = footsteps.output_dir + name + "/"
    #os.mkdir(output_path)
    with open(davis_path + "/ImageSets/2017/val.txt", "r") as f:
        sequences = f.readlines()
    for sequence in tqdm.tqdm(sequences[4:5]):
        sequence = sequence[:-1]  # strip newline
        sequence_out_path = output_path + sequence + "/"
        sequence_img_path = davis_path + "JPEGImages/480p/" + sequence + "/"
        with open(davis_path + "Annotations/480p/" + sequence + "/00000.png", "rb") as handle:
          first_annotation = np.array(
            Image.open(handle)
          )

        #os.mkdir(output_path + sequence)
        with open(sequence_img_path + "00000.jpg", "rb") as handle:
          first_image = np.array(Image.open(handle))

        prev_image = first_image
        prev_annotation = first_annotation

        for i in range(1, len(os.listdir(sequence_img_path)))[::5]:
            with open(sequence_img_path + f"{i:05}.jpg", "rb") as handle:
              curr_image = np.array(Image.open(handle))
            annotation = model(
                first_image, first_annotation, prev_image, prev_annotation, curr_image
            )
            Image.fromarray(annotation).save(sequence_out_path + f"{i:05}.png")

            prev_image = curr_image
            prev_annoration = first_annotation
    return
    subprocess.run(
        [
            "python",
            "davis2017-evaluation/evaluation_method.py",
            "--davis_path",
            "data_storage/DAVIS",
            "--results_path",
            output_path,
            "--task",
            "semi-supervised",
        ]
    )


In [13]:
from tensorflow_.models import make_model
from tensorflow_.fmapicon_utils import execute_model
import matplotlib.pyplot as plt
l = []

class FMAPICON_model:
    def __init__(self, weights_path):
        self.inner_model = make_model(.6, 90, 128)
        self.inner_model.load_weights(weights_path)
    def __call__(self, initial_frame, initial_mask, prev_frame, prev_mask, current_frame):
        crop = (initial_frame.shape[1] - initial_frame.shape[0]) // 2
        B = initial_frame[None, ::4, crop:-crop:4] / 255.   
        A = current_frame[None, ::4, crop:-crop:4] / 255.
        
        cc, grid = execute_model(A, B, self.inner_model, N=1)
        m = initial_mask[::4, crop:-crop:4]

        shitty_res = m[grid[0, :, :, 1].astype(int) + 15, grid[0, :, :, 0].astype(int) + 15].transpose()

        up_res = np.repeat(np.repeat(shitty_res, 4, axis=0),4, axis=1)
        
        out_mask = initial_mask.copy()
        
        out_mask[15 * 4:-15 * 4, crop + 15 * 4:-crop - 15 * 4] = up_res
        
        plt.imshow(A[0, 15:-15, 15:-15])
        plt.show()
        #plt.imshow(B[0, 15:-15, 15:-15])
        #plt.show()
        plt.imshow(out_mask)
        plt.show()
        l.append(up_res)
        
        plt.imshow(grid[0, :, :, 0].transpose())
        plt.show()
        
        l.append((cc, A, B, initial_mask))
        
        

        return initial_mask

#DavisEval(FMAPICON_model("tensorflow_/results/log_probability_2/epoch50/model_weights.tf"), "FMAPICON")
DavisEval(FMAPICON_model("tensorflow_/results/clip.6_11/epoch51/model_weights.tf"), "FMAPICON")

  0%|                                                                                                                                             | 0/1 [00:00<?, ?it/s]


AttributeError: 'Tensor' object has no attribute 'cast'

In [6]:
grid,A,B,m=l[1]

IndexError: list index out of range

In [ ]:
grid.shape

In [ ]:
plt.imshow(grid[0, :, :, 0].transpose())
plt.show()
plt.imshow(A[0])
plt.show()
plt.imshow(B[0])
plt.show()

In [ ]:
shitty_res = m[grid[0, :, :, 1].astype(int) + 15, grid[0, :, :, 0].astype(int) + 15].transpose()

up_res = np.repeat(np.repeat(shitty_res, 4, axis=0),4, axis=1)

In [ ]:
m.shape

In [ ]:
plt.imshow(m[15:-15, 15:-15])

In [ ]:
len(l)